### Exploring Ebay Car Sales Data

#### Aim of the project: Clean the data and analyze the used car listings on ebay (a reduced dataset to make fast analysis but with valid conclusions). 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re

autos = pd.read_csv("autos.csv",encoding = "Windows-1252")

In [2]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [3]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

In [4]:
autos.shape

(50000, 20)

##### FIrst insights: 

- The columns "fuelType", "notRepairedDamage", "model", "gearbox" and "vehicleType" have null values (all of them less than 20% of null values). 

- The date columns "lastSeen", "dateCreated" and "dateCrawled" are object types but could be datetime object.

- The "odometer" and "price" columns are object type but could be numerical. 

- The column names are in CamelCase, but is preferred to be snake_case

In [5]:
old_columns_names = autos.columns

In [6]:
def camel_to_snake_case(word):
# Takes in a word in CamelCase
# and return the same word as snake_case
    from string import ascii_letters
    mayúsculas = ascii_letters[-26:]
    minúsculas = ascii_letters[:26]
    new_word = word
    list_letter_index = []
    for index,letter in enumerate(new_word):
        if letter in mayúsculas:
            list_letter_index.append((index,letter))
            # print(index,letter)
    # print(list_letter_index)
    cont = 0
    for i in list_letter_index:   
        new_word = new_word[:i[0]+cont]+"_"+i[1].lower()+new_word[i[0]+cont+1:]
        cont += 1
    return new_word

new_columns_names = old_columns_names

for i in range(len(old_columns_names)):
    old_column_name = old_columns_names[i]
    new_column_name = camel_to_snake_case(old_column_name)
    new_columns_names = new_columns_names.str.replace(old_column_name,new_column_name)

print(new_columns_names)

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'year_of_registration', 'gearbox', 'power_p_s', 'model',
       'odometer', 'month_of_registration', 'fuel_type', 'brand',
       'not_repaired_damage', 'date_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


In [7]:
final_new_columns_names = new_columns_names.str.replace("power_p_s","power_ps")

In [8]:
print(final_new_columns_names)

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'year_of_registration', 'gearbox', 'power_ps', 'model',
       'odometer', 'month_of_registration', 'fuel_type', 'brand',
       'not_repaired_damage', 'date_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


In [9]:
autos.columns = final_new_columns_names

In [10]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'year_of_registration', 'gearbox', 'power_ps', 'model',
       'odometer', 'month_of_registration', 'fuel_type', 'brand',
       'not_repaired_damage', 'date_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

In [11]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,year_of_registration,gearbox,power_ps,model,odometer,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [12]:
autos.describe(include="all")

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,year_of_registration,gearbox,power_ps,model,odometer,month_of_registration,fuel_type,brand,not_repaired_damage,date_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-27 22:55:05,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


##### New observation: 

- The columns "seller" and "offer_type" have mostly one value: "privat" and "Angebot" respectively. These columns are candidates to be dropped. 

- Also we can change the type of the columns "price" and "odometer" to integers. Let's do that:

In [13]:
autos["price"] = autos["price"].str.replace(",","")
autos["price"] = autos["price"].str.replace("$","")
autos["price"] = autos["price"].astype(int)

autos["odometer"] = autos["odometer"].str.replace(",","")
autos["odometer"] = autos["odometer"].str.replace("km","")
autos["odometer"] = autos["odometer"].astype(int,copy=False)

autos.rename({"odometer":"odometer_km",\
              "not_repaired_damage":"unrepaired_damage",\
              "year_of_registration":"registration_year",\
             "month_of_registration":"registration_month"},\
             axis=1,inplace=True)
print(autos.columns)

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'date_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')


In [14]:
# Verification that the columns "price" and "odometer" 
# (also we changed his name to odometer_km) has changed 
# to type in64

autos.dtypes

date_crawled          object
name                  object
seller                object
offer_type            object
price                  int64
abtest                object
vehicle_type          object
registration_year      int64
gearbox               object
power_ps               int64
model                 object
odometer_km            int64
registration_month     int64
fuel_type             object
brand                 object
unrepaired_damage     object
date_created          object
nr_of_pictures         int64
postal_code            int64
last_seen             object
dtype: object

In [15]:
print("Unique values from odometer_km column:", autos["odometer_km"].unique().shape[0])
print("Unique values from price column:",autos["price"].unique().shape[0])

Unique values from odometer_km column: 13
Unique values from price column: 2357


#### <i> Let's do some statistics applying the describe() and value_counts() methods to see the behavior of the "odometer_km" column. <i\>

In [16]:
autos["odometer_km"].describe()

count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64

In [17]:
autos["odometer_km"].value_counts().sort_index(ascending=False)

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
40000       819
30000       789
20000       784
10000       264
5000        967
Name: odometer_km, dtype: int64

#### <i> Let's do the same we do with "odometer_km" column to "price" column.

In [18]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [19]:
autos["price"].value_counts().sort_index(ascending=False).head(30)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
194000      1
190000      1
180000      1
175000      1
169999      1
169000      1
163991      1
163500      1
155000      1
151990      1
Name: price, dtype: int64

In [20]:
autos["price"].value_counts().head(30)

0       1421
500      781
1500     734
2500     643
1000     639
1200     639
600      531
800      498
3500     498
2000     460
999      434
750      433
900      420
650      419
850      410
700      395
4500     394
300      384
2200     382
950      379
1100     376
1300     371
3000     365
550      356
1800     355
5500     340
1250     335
350      335
1600     327
1999     322
Name: price, dtype: int64

#### Seems that there are some outliers with extremly high and extremly lows values.

Let's cut the price column between USD 50 and USD 1M dollars (1M from 1 million)

In [21]:
autos[autos["price"].between(50,1000000)]["price"].describe()

count     48350.000000
mean       5977.136381
std       11983.401860
min          50.000000
25%        1250.000000
50%        3000.000000
75%        7499.000000
max      999999.000000
Name: price, dtype: float64

In [22]:
autos[autos["price"].between(50,1000000)]["price"].\
value_counts().head(21)

500     781
1500    734
2500    643
1000    639
1200    639
600     531
3500    498
800     498
2000    460
999     434
750     433
900     420
650     419
850     410
700     395
4500    394
300     384
2200    382
950     379
1100    376
1300    371
Name: price, dtype: int64

#### Let's do some analysis of dates in date_crawled, last_seen, and ad_created columns in date timestamp format. 

In [23]:
# DATE CRAWLED
autos["date_crawled"].str[:10].\
value_counts(normalize=True,dropna=False).\
sort_index(ascending=True)

2016-03-05    0.02538
2016-03-06    0.01394
2016-03-07    0.03596
2016-03-08    0.03330
2016-03-09    0.03322
2016-03-10    0.03212
2016-03-11    0.03248
2016-03-12    0.03678
2016-03-13    0.01556
2016-03-14    0.03662
2016-03-15    0.03398
2016-03-16    0.02950
2016-03-17    0.03152
2016-03-18    0.01306
2016-03-19    0.03490
2016-03-20    0.03782
2016-03-21    0.03752
2016-03-22    0.03294
2016-03-23    0.03238
2016-03-24    0.02910
2016-03-25    0.03174
2016-03-26    0.03248
2016-03-27    0.03104
2016-03-28    0.03484
2016-03-29    0.03418
2016-03-30    0.03362
2016-03-31    0.03192
2016-04-01    0.03380
2016-04-02    0.03540
2016-04-03    0.03868
2016-04-04    0.03652
2016-04-05    0.01310
2016-04-06    0.00318
2016-04-07    0.00142
Name: date_crawled, dtype: float64

In [24]:
autos["date_crawled"].str[:10].\
value_counts(normalize=True,dropna=False).\
sort_index(ascending=True).describe()

count    34.000000
mean      0.029412
std       0.009765
min       0.001420
25%       0.029885
50%       0.032710
75%       0.034885
max       0.038680
Name: date_crawled, dtype: float64

<i> The date crawled ranges from 2016-03-05 to 2016-04-07 (1 month). Almost all the dates has around of 3% of contribution to the sample.

In [25]:
# LAST SEEN
autos["last_seen"].str[:10].\
value_counts(normalize=True,dropna=False).\
sort_index(ascending=True)

2016-03-05    0.00108
2016-03-06    0.00442
2016-03-07    0.00536
2016-03-08    0.00760
2016-03-09    0.00986
2016-03-10    0.01076
2016-03-11    0.01252
2016-03-12    0.02382
2016-03-13    0.00898
2016-03-14    0.01280
2016-03-15    0.01588
2016-03-16    0.01644
2016-03-17    0.02792
2016-03-18    0.00742
2016-03-19    0.01574
2016-03-20    0.02070
2016-03-21    0.02074
2016-03-22    0.02158
2016-03-23    0.01858
2016-03-24    0.01956
2016-03-25    0.01920
2016-03-26    0.01696
2016-03-27    0.01602
2016-03-28    0.02086
2016-03-29    0.02234
2016-03-30    0.02484
2016-03-31    0.02384
2016-04-01    0.02310
2016-04-02    0.02490
2016-04-03    0.02536
2016-04-04    0.02462
2016-04-05    0.12428
2016-04-06    0.22100
2016-04-07    0.13092
Name: last_seen, dtype: float64

In [26]:
autos["last_seen"].str[:10].\
value_counts(normalize=True,dropna=False).\
sort_index(ascending=True).describe()

count    34.000000
mean      0.029412
std       0.043483
min       0.001080
25%       0.012590
50%       0.019380
75%       0.023835
max       0.221000
Name: last_seen, dtype: float64

The last seen column also ranges from 2016-03-05 to 2016-04-07 (1 month). 

#### Now we can see that the behavior of the values in last_seen columns change.

The mean is almost 3% but its no representative because the followings are outliers:
2016-04-05    0.12428
2016-04-06    0.22100
2016-04-07    0.13092

Apparently on April 7th they produced the 22.1% of the "last seen" of the ad. This is like 10 times most frequent that the other 31 dates. 


In [27]:
#DATE CREATED 
autos["date_created"].str[:10].\
value_counts(normalize=True,dropna=False).\
sort_index(ascending=True).head(30)

2015-06-11    0.00002
2015-08-10    0.00002
2015-09-09    0.00002
2015-11-10    0.00002
2015-12-05    0.00002
2015-12-30    0.00002
2016-01-03    0.00002
2016-01-07    0.00002
2016-01-10    0.00004
2016-01-13    0.00002
2016-01-14    0.00002
2016-01-16    0.00002
2016-01-22    0.00002
2016-01-27    0.00006
2016-01-29    0.00002
2016-02-01    0.00002
2016-02-02    0.00004
2016-02-05    0.00004
2016-02-07    0.00002
2016-02-08    0.00002
2016-02-09    0.00004
2016-02-11    0.00002
2016-02-12    0.00006
2016-02-14    0.00004
2016-02-16    0.00002
2016-02-17    0.00002
2016-02-18    0.00004
2016-02-19    0.00006
2016-02-20    0.00004
2016-02-21    0.00006
Name: date_created, dtype: float64

In [28]:
autos["date_created"].str[:10].\
value_counts(normalize=True,dropna=False).\
sort_index(ascending=True).describe()

count    76.000000
mean      0.013158
std       0.015958
min       0.000020
25%       0.000020
50%       0.000140
75%       0.032275
max       0.038920
Name: date_created, dtype: float64

The date crawled ranges from 2015-06-11 to 2016-04-07 (almost 1 year). 

In the case of "date created" there are to low percentages until the 75th (not included). After the 75th percentile wich increse the rate up to 3.2% close to the max value of 3.9%.

In [29]:
#REGISTRATION YEAR OF THE CAR
autos["registration_year"].describe()

count    50000.000000
mean      2005.073280
std        105.712813
min       1000.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

#### Again seems that bad data is included, there can't be a car registered in the year 1000 or 9999. Clearly these are errors in the data. Debido a esto podemos establecer los dos límites de años de registro de una manera razonable:

- El primer automóvil no puede registrarse antes de que comience la industria del automóvil, alrededor del 1900.
- El último automóvil registrado no puede exceder el año de 2016 porque no se puede hacer un registro de un automóvil sin haberlo visto aún.

In [30]:
autos[autos["registration_year"].\
between(1900,2016)]["registration_year"].\
value_counts(normalize=True).head(30)

2000    0.069834
2005    0.062776
1999    0.062464
2004    0.056988
2003    0.056779
2006    0.056384
2001    0.056280
2002    0.052740
1998    0.051074
2007    0.047972
2008    0.046452
2009    0.043683
1997    0.042225
2011    0.034022
2010    0.033251
1996    0.030066
2012    0.027546
2016    0.027401
1995    0.027338
2013    0.016782
2014    0.013867
1994    0.013742
1993    0.009265
2015    0.008308
1990    0.008224
1992    0.008141
1991    0.007412
1989    0.003769
1988    0.002957
1985    0.002186
Name: registration_year, dtype: float64

In [31]:
autos[autos["registration_year"].\
between(1900,2016)]["registration_year"].\
value_counts(normalize=True).sort_index(ascending=True).head(30)

1910    0.000187
1927    0.000021
1929    0.000021
1931    0.000021
1934    0.000042
1937    0.000083
1938    0.000021
1939    0.000021
1941    0.000042
1943    0.000021
1948    0.000021
1950    0.000062
1951    0.000042
1952    0.000021
1953    0.000021
1954    0.000042
1955    0.000042
1956    0.000104
1957    0.000042
1958    0.000083
1959    0.000146
1960    0.000708
1961    0.000125
1962    0.000083
1963    0.000187
1964    0.000250
1965    0.000354
1966    0.000458
1967    0.000562
1968    0.000541
Name: registration_year, dtype: float64

##### First look on registration year column:

- We can see that in the middle of the 90's the registration of cars accelerate up to 2005. One possible explanation is the explosion of electronic commerce in that year. 

- The reason for the decline in the records after 2005 (but still a high percentage) may be that the large number of cars registered since the 94s were re-sold on the same platform very effectively. All this could had an impact on the sales of new cars.  

#### Now we can do some analysis in the brand column:

In [32]:
autos["brand"].unique()

array(['peugeot', 'bmw', 'volkswagen', 'smart', 'ford', 'chrysler',
       'seat', 'renault', 'mercedes_benz', 'audi', 'sonstige_autos',
       'opel', 'mazda', 'porsche', 'mini', 'toyota', 'dacia', 'nissan',
       'jeep', 'saab', 'volvo', 'mitsubishi', 'jaguar', 'fiat', 'skoda',
       'subaru', 'kia', 'citroen', 'chevrolet', 'hyundai', 'honda',
       'daewoo', 'suzuki', 'trabant', 'land_rover', 'alfa_romeo', 'lada',
       'rover', 'daihatsu', 'lancia'], dtype=object)

In [33]:
autos["brand"].value_counts().head(20)

volkswagen        10687
opel               5461
bmw                5429
mercedes_benz      4734
audi               4283
ford               3479
renault            2404
peugeot            1456
fiat               1308
seat                941
skoda               786
mazda               757
nissan              754
smart               701
citroen             701
toyota              617
sonstige_autos      546
hyundai             488
volvo               457
mini                424
Name: brand, dtype: int64

In [34]:
brands = autos["brand"].value_counts().head(20).index.tolist()

#### The brands have been classified in number of ads. Then, in the list of brands, there are 20 top brands.


In [36]:
# the price bounds for this analysis are chosen between (500,500.000)
grouped_brands_prices = {}
autos_reasonable_prices = autos[autos["price"].\
    between(500,500000)]
for brand in brands:
    grouped_brands_prices[brand] = autos_reasonable_prices\
    [autos_reasonable_prices["brand"]==brand]\
    ["price"].mean()
    
print(grouped_brands_prices)

average_price_brand = pd.DataFrame(columns = ["brand","avg_price"])
i=0
for k,v in grouped_brands_prices.items():
    average_price_brand.loc[i,"brand"] = k
    average_price_brand.loc[i,"avg_price"] = v
    i += 1

{'volkswagen': 5701.241955574009, 'opel': 3348.936054124836, 'bmw': 8514.81103286385, 'mercedes_benz': 8670.394058540847, 'audi': 9484.499876390606, 'ford': 4267.123376623376, 'renault': 2760.8114143920598, 'peugeot': 3329.3254211332314, 'fiat': 3211.992606284658, 'seat': 4709.527446300716, 'skoda': 6539.7107001321, 'mazda': 4391.736377025037, 'nissan': 5077.141592920354, 'smart': 3552.050946142649, 'citroen': 3994.679626749611, 'toyota': 5244.499165275459, 'sonstige_autos': 13200.66974595843, 'hyundai': 5695.464757709251, 'volvo': 5110.575539568345, 'mini': 10616.96626506024}


In [37]:

average_price_brand.sort_values(by="avg_price",ascending=False)

,brand,avg_price
16,sonstige_autos,13200.7
19,mini,10617
4,audi,9484.5
3,mercedes_benz,8670.39
2,bmw,8514.81
10,skoda,6539.71
0,volkswagen,5701.24
17,hyundai,5695.46
15,toyota,5244.5
18,volvo,5110.58


In [39]:
# there aren't bounds for km since the column
# has reasonable values

grouped_brands_mileage = {}
autos_reasonable_km = autos
for brand in brands:
    grouped_brands_mileage[brand] = autos_reasonable_km\
    [autos_reasonable_km["brand"]==brand]\
    ["odometer_km"].mean()
    
print(grouped_brands_mileage)


bmp_series = pd.Series(grouped_brands_prices)
bmk_series = pd.Series(grouped_brands_mileage)

df = pd.DataFrame(bmp_series,columns=["brand_mean_prices"])
df["brand_mean_mileage"] = bmk_series

df

{'volkswagen': 128955.27276129878, 'opel': 129298.66324848929, 'bmw': 132521.64302818198, 'mercedes_benz': 130886.14279678918, 'audi': 129643.9411627364, 'ford': 124131.93446392642, 'renault': 128223.79367720465, 'peugeot': 127352.33516483517, 'fiat': 117037.4617737003, 'seat': 122061.63655685441, 'skoda': 110947.83715012722, 'mazda': 125132.10039630119, 'nissan': 118978.7798408488, 'smart': 100756.06276747503, 'citroen': 119764.62196861627, 'toyota': 115988.65478119935, 'sonstige_autos': 87188.6446886447, 'hyundai': 106782.7868852459, 'volvo': 138632.3851203501, 'mini': 89375.0}


,brand_mean_prices,brand_mean_mileage
audi,9484.499876,129643.941163
bmw,8514.811033,132521.643028
citroen,3994.679627,119764.621969
fiat,3211.992606,117037.461774
ford,4267.123377,124131.934464
hyundai,5695.464758,106782.786885
mazda,4391.736377,125132.100396
mercedes_benz,8670.394059,130886.142797
mini,10616.966265,89375.000000
nissan,5077.141593,118978.779841


#### Prices and Odometer: 

<i> Seems that while we looking only in the ranking of prices we can conclude incorrectly that sonstige_autos with an average price of 13.201 dollars they have an average mileage of 87.188 km.<i\>

Instead Mercedes Benz for example has an average price of 8670 dollars (less than sonstige_autos) but an average mileage of 130886 km. We need to go further in the analysis and looks for any correlations. But for now is enought for a basic analysis. 
<i\>

In [40]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'date_created', 'nr_of_pictures', 'postal_code',
       'last_seen'],
      dtype='object')

##### Finding the most common brand/model combination:

In [41]:
grouped_brands_models = {}

for brand in brands:
    try:
        brand_model = autos.loc[autos["brand"]==brand]["model"].\
    value_counts().index[0]
        grouped_brands_models[brand]=brand_model
    except: continue
        
mb_series = pd.Series(grouped_brands_models)
mb_series

audi                   a4
bmw                   3er
citroen            andere
fiat                punto
ford                focus
hyundai           i_reihe
mazda             3_reihe
mercedes_benz    c_klasse
mini               cooper
nissan              micra
opel                corsa
peugeot           2_reihe
renault            twingo
seat                ibiza
skoda             octavia
smart              fortwo
toyota              yaris
volkswagen           golf
volvo                 v70
dtype: object